### 10.1.1 Basic Conditional Evaluation ( $ )

**$[exprcond; exprtrue; exprfalse]**

Here exprcond is an expression that evaluates to a boolean atom. Analogous to C, the result of exprcond can be any type whose underlying value is an integer. The result of the conditional is the evaluation of exprtrue when exprcond is not zero and exprfalse if it is zero.

In [1]:
$[1b;42;9*6]

42


Although evaluation of function arguments in q is eager, evaluation of the expressions in the conditional is short circuited, meaning that only the one selected for return is evaluated. Again in a fresh q session,

In [1]:
$[1b;a:42;b:43]
a
b

42


42


[0;31mb[0m: [0;31mb[0m

Observe that a test for zero in exprcond is redundant: remove that test and reverse the order of the second and third arguments.

In [3]:
z:0
$[z=0;1.1;-1.1]
$[z;-1.1;1.1]

1.1


1.1


### 10.1.2 Extended Conditional Evaluation ( $ )

**Sequence of expressions rather than 1 expression**

A similar effect can be achieved in q using an extended form of conditional evaluation with $.

$[exprcond; [exprtrue1; …]; [exprfalse1; …]]

In [4]:
v:42
$[v=42; [a:6;b:7;`Everything]; [a:`Life;b:`the;c:`Universe;a,b,c]]

`Everything


**cascading form of if-else in which multiple tests  are made**

A similar effect can be achieved in q with another extended form of conditional execution.

$[exprcond1;exprtrue1; …;exprcondn;exprtruen;exprfalse]

Any condition other than the first is only evaluated if all those prior to it have evaluated to zero. Otherwise put, a condition evaluating to non-zero short-circuits the evaluation of subsequent ones.

**multi-level conditions**

Finally, the previous extended form of conditional execution can be further extended by substituting a bracketed sequence of expressions for any exprtrue or exprfalse.


$[expr_cond1;[expr_true11; …]; …;
    expr_condn;[expr_truen1; …];
    [expr_false1; …]]

### 10.1.3 Vector Conditional Evaluation ( ? )

#### Triadic vector-conditional evaluation ? has the form,

?[vb;exprtrue;exprfalse]

where vb is a simple boolean list and exprtrue and exprfalse are of the same type and are either atoms or vectors that conform to vb. The result conforms to vb and selects from exprtrue in positions where vb is 1b and exprfalse in positions where vb has 0b. **All arguments of vector-conditional are fully executed. In other words, there is no short circuiting of evaluation.**

In [13]:
L:til 10
L
L mod 3

0 1 2 3 4 5 6 7 8 9


0 1 2 0 1 2 0 1 2 0


In [14]:
?[0<>L mod 3; L; 42]

42 1 2 42 4 5 42 7 8 42


Vector conditional is especially useful with table columns.

In [15]:
t:([] c1:1.1 2.2 3.3; c2:10 20 30; c3:100 200 300)
update mix:?[c1>2.0; c3; c2] from t

c1  c2 c3  mix
--------------
1.1 10 100 10 
2.2 20 200 200
3.3 30 300 300


### 10.1.4 if

The imperative if statement conditionally evaluates a sequence of expressions. It is not a function and does not return a value. It has the form,

if[exprcond;expr1; …;exprn]

The exprcond is evaluated and if it is non-zero the expressions expr1 thru exprn are evaluated in left-to-right order. As with other conditionals, the brackets do not create lexical scope, so variables defined in the body exist in the same scope as the if.

In [ ]:
Here is an example that creates two global variables and modifies one.

In [16]:
a:42
b:98.6

In [17]:
if[a=42;x:6;y:7;b:a*b]

In [19]:
x
y
b

6


7


4141.2


**Well-written q code rarely needs if. One example of legitimate use is pre-checking function arguments to abort execution for bad values.**

### 10.1.5 do ( ~for)

The imperative do statement allows repeated execution of a block of statements. It has the form,

do[exprcount;expr1; …;exprn]

where exprcount must evaluate to an non-negative integer. The expressions expr1 thru exprn are evaluated exprcount times in left-to-right order. Note that do is a statement, not a function, and does not have an explicit result.



The following expression is a loopy computation of n factorial. It iterates n - 1 times, decrementing the factor f on each pass.

In [20]:
n:5
do[-1+f:r:n; r*:f-:1] / do not do this!
r

120


**The best recommendation about usage of do is: Don’t!**

The only legitimate use of do that the author has encountered is to time the execution of a q expression that runs too quickly for the timer to get an accurate reading, but this has been obviated by the enhanced \t command.

In [21]:
\t v*v:til 1000000
\t do[100; v*v:til 1000000]
\t:100 v*v:til 1000000

8


329


337


### 10.1.6 while

The imperative while statement is an iterator of the form,

while[exprcond;expr1; …;exprn]

where exprcond is evaluated and the expressions expr1 thru exprn are evaluated repeatedly in left-to-right order as long as exprcond is non-zero. The while statement is not a function, does not have an explicit result and does not introduce lexical scope.

Here is loopy factorial redone with while.

In [22]:
f:r:n:5

In [23]:
while[f-:1;r*:f] / do not do this either!
r

120


#### 10.1.7 Return and Signal

Normal function application evaluates each expression in the function body in sequence and terminates after the last one. There are two mechanisms for ending the execution early: 
- one indicates successful completion 
- and the other signals abrupt termination.

**To terminate function application immediately and return a normal value, use an empty assignment – that is, : with the return value to its right and no variable to its left.** 

For example, in the following instrumented function, application is terminated and the result is returned in the fourth expression. The final expression is never evaluated.

In [24]:
f:{0N!"Begin"; a:x; b:y; :a*b; "End"}
f[4;5]

"Begin"


20


**To abort function execution immediately with an exception, use signal, which is single-quote ', with an error message to its right.** The error message can be provided as a symbol or string.

You too can return pithy error messages. For example, in the following function, execution will be aborted in the fourth expression. The final expression that assigns c is never evaluated.

In [24]:
g:{0N!"Begin"; a:x; b:y; '"End"; c:b}
g[4;5]

"Begin"


[0;31mEnd[0m: [0;31mEnd[0m

A legitimate use of the if statement is to terminate execution with an exception. The following snippet would typically reside inside a function body.

In [24]:
if[a<50; '"Bad a"];

[0;31mBad a[0m: [0;31mBad a[0m

#### 10.1.8 Protected Evaluation ( ~ try-catch)

Q provides a try-catch capability using triadic forms of function application. Triadic @ is used for monadic functions and triadic . is used for multivalent functions. The syntax is the same for both.

@[fmon;a;exprfail]

.[fmul;Largs;exprfail]

Here fmon is a monadic function, a is single argument, fmul is a multivalent function, Largs is a list of arguments, and exprfail is an expression or function. In both forms, the function is applied to its argument(s). Upon successful application, protected evaluation returns the result of the application. Should an exception arise, exprfail is applied to the resulting error string.

You can use protected evaluation to log error messages from exceptions that would otherwise crash your program.

In [25]:
prod:{x*y}
.[prod; (6;7); show]
.[prod; (6;`7); show]

42


"type"


### 10.2 Debugging

#### Simplest technique

Let’s get real. **Say you want to set a breakpoint in your q program. Easy: just insert a line that you know will fail – use an undefined name.** For example, to pause execution before the last expression in the function f below, insert any undefined name there – “break” is commonly used.

In [0]:
f:{a:x*x; b:y*y; a+b}
f:{a:x*x; b:y*y; break; a+b}
f[3;4]

[0;31mbreak[0m: [0;31mbreak[0m

**In a q session, you can tell that execution has been suspended by the extra parenthesis at the q prompt.** At this point, you have the full power of the q console available to inspect the current state of your program.

**Once you have finished your inspection and debugging, you should either return from the function with a value or abort execution using \ .** In either case, the extra ) at the q prompt will disappear.

:
or
\

#### More complex technique

A slightly more sophisticated technique allows you to continue execution after the break. Here we cause the break one level lower. A forced return entered at the console completes the breakpoint execution and continues execution of f.

(Examples work in real q console, not sure how is this different from the simplest one)

In [0]:
breakpoint:{break}
f:{a:x*x; b:y*y; breakpoint[]; a+b}
f[3;4]

[0;31mbreak[0m: [0;31mbreak[0m

#### debug zs function

In [1]:
zs:{`d`P`L`G`D!(system"d"),v[1 2 3],enlist last v:value x}

This function takes another function as its argument and returns a dictionary with entries for the current directory, function parameters, local variables referenced, global variables referenced and the function definition.

In [3]:
b:7
f:{a:6; x+a*b}
zs f

d| `.
P| ,`x
L| ,`a
G| ``b
D| "{a:6; x+a*b}"


A good place to start with zs when you have suspended execution is with the system variable .z.s that holds the suspended function itself.

### 10.3 Scripts

A script is a q program stored in a text file with an extension of .q (or .k if you are writing k code). A script can contain any q expressions or commands. The contents of the script are parsed and evaluated sequentially from top to bottom. Global entities created during execution of the script exist in the workspace after the script is loaded and executed.

#### 10.3.1 Creating and Loading a Script (\l)

A script can be loaded at any time during a session using the \l command, called load.

\l /q4m/trades.q

You can have q load a script on startup by placing its name after the call to the q executable on the operating system command line.

$q /q4m/trades.q

#### 10.3.2 Blocks

You can comment out a block of code (i.e., multiple lines) in a script by surrounding it with matching / and \ with each at the beginning of its own line. 

An unmatched \ at the beginning of a line exits the script.

In [4]:
/
sadsddsa
afsadas
addm
\
a:5
a

5


**Multi-line expressions are permitted in a script but they have a special form.**

- The first line must not be indented – i.e., it begins at the left of the line with no initial whitespace.
- Any continuation lines must be indented, meaning that there is at least one whitespace character at the beginning of the line.
- In particular, if you put the closing brace to a function definition on its own line, it must be indented. Do not use the common C style of aligning the closing brace with the function name.
- Empty lines and comment lines (beginning with /) are permitted anywhere.
- Table definition and function definition provide nice opportunities for splitting across multiple lines:



- A table can have line breaks after a closing square bracket ] or after a semicolon separator ;
- A function can have line breaks after a closing square bracket ] or after a comma separator ,.

#### 10.3.3 Passing Parameters

Parameters are passed to a q script at q startup similarly to argv command line parameters in C. Specifically, the system variable **.z.x** comprises a list of strings, each containing the character representation of an argument present on the command line that invoked the script. 

In [10]:
mktrades:{[tickers; sz]
  dt:2015.01.01+sz?31;
  tm:sz?24:00:00.000;
  sym:sz?tickers;
  qty:10*1+sz?1000;
  px:90.0+(sz?2001)%100;
  t:([] dt; tm; sym; qty; px);
  t:`dt`tm xasc t;
  t:update px:6*px from t where sym=`goog;
  t:update px:2*px from t where sym=`ibm;
  t}

size:"I"$.z.x 0

trades:mktrades[`aapl`goog`ibm; size]

q /q4m/trades.q 2000000

As of this writing (Sep 2015), parameters can be passed when a script is loaded at q startup but not when a script is loaded with \l or system “l”.